In [1]:
from pathlib import Path
import os

from bids import BIDSLayout
import numpy as np
import mne
from pyclustering.cluster.kmeans import kmeans
from pyclustering.cluster.center_initializer import random_center_initializer
from pyclustering.utils.metric import distance_metric, type_metric
import pandas as pd
from scipy.stats import wasserstein_distance
from scipy.spatial.distance import euclidean
import ot

In [2]:
bids_root = Path(os.environ['biomag2020_data-bids'])

In [3]:
layout = BIDSLayout(bids_root, validate=True, derivatives=True)
layout.derivatives

{'01_preprocessing': BIDS Layout: ...s\derivatives\01_preprocessing | Subjects: 33 | Sessions: 66 | Runs: 0,
 '02_eigenvalues': BIDS Layout: ...ids\derivatives\02_eigenvalues | Subjects: 33 | Sessions: 66 | Runs: 0}

In [4]:
eigenvalue_files = layout.get(suffix='eigenvalues', extension='npy')

In [5]:
subjects = [f.entities['subject'] for f in eigenvalue_files]
sessions = [f.entities['session'] for f in eigenvalue_files]

In [6]:
df = (
    pd.DataFrame(
        columns=['subject', 'session'],
        data=zip(subjects, sessions))
    .reset_index()
    .rename(columns=dict(index='eigs_id')))
df.head()

,eigs_id,subject,session
0,0,BQBBKEBX,1457629800
1,1,BQBBKEBX,1458832200
2,2,BYADLMJH,1416503760
3,3,BYADLMJH,1417706220
4,4,CECMHHYP,1364481360


In [7]:
eigs_all = np.stack([np.load(f) for f in eigenvalue_files])
eigs_all = eigs_all.clip(0, 2)

In [8]:
n_bins = 100
bins = np.quantile(eigs_all.flatten(), np.linspace(0, 1, n_bins + 1))
bin_centers = (bins[:-1] + bins[1:]) / 2

Matrix of distances between bins

In [9]:
M = np.asarray([[abs(bc2 - bc1) for bc1 in bin_centers] for bc2 in bin_centers])
M /= M.max()

Convert samples to pmfs

In [10]:
histograms = np.asarray([np.histogram(spectrum, bins=bins)[0] for spectrum in eigs_all])
pmfs = (histograms.T / histograms.sum(axis=1)).T

## k-means

In [11]:
class KMeans(kmeans):
    def _kmeans__update_centers(self):
        """!
        @brief Calculate centers of clusters in line with contained objects.

        @return (numpy.array) Updated centers.

        """
        numpy = np
        
        dimension = self._kmeans__pointer_data.shape[1]
        centers = numpy.zeros((len(self._kmeans__clusters), dimension))

        for index in range(len(self._kmeans__clusters)):
            cluster_points = self._kmeans__pointer_data[self._kmeans__clusters[index], :]
            # centers[index] = cluster_points.mean(axis=0)
            centers[index] = ot.barycenter(cluster_points.T, M=self.M, reg=self.reg)
            
        return numpy.array(centers)

In [12]:
k = 2

In [13]:
initial_centers = random_center_initializer(pmfs, k, random_state=3).initialize()
wasserstein_metric = distance_metric(type_metric.USER_DEFINED, 
                                     func=lambda x, y: wasserstein_distance(bin_centers, bin_centers, x, y))
kmeans_instance = KMeans(data=pmfs, initial_centers=initial_centers, metric=wasserstein_metric)

kmeans_instance.M = M
kmeans_instance.reg = 0.05

In [14]:
kmeans_instance.process()
final_centers = kmeans_instance.get_centers()
clusters = kmeans_instance.get_clusters()

In [15]:
cluster_assignment = pd.DataFrame(columns=['cluster_id', 'eigs_id'],
             data=[(cluster_id, eigs_id) 
                     for cluster_id, cluster in enumerate(clusters, 1)
                     for eigs_id in cluster
                    ]
            ).sort_values(by='eigs_id').reset_index(drop=True)
cluster_assignment.head()

,cluster_id,eigs_id
0,2,0
1,2,1
2,1,2
3,2,3
4,1,4


Number of spectra per cluster.

In [16]:
[len(c) for c in clusters]

[17, 49]

Number of subjects with distinct combinations of session clustering.

In [17]:
df2 = df.merge(cluster_assignment, on='eigs_id')
df2.head()

,eigs_id,subject,session,cluster_id
0,0,BQBBKEBX,1457629800,2
1,1,BQBBKEBX,1458832200,2
2,2,BYADLMJH,1416503760,1
3,3,BYADLMJH,1417706220,2
4,4,CECMHHYP,1364481360,1


In [18]:
df2.groupby('subject').agg(dict(cluster_id=['min', 'max'])).value_counts().sort_index()

(cluster_id, min)  (cluster_id, max)
1                  1                     2
                   2                    13
2                  2                    18
dtype: int64

Number of subject with sessions clustered into two clusters vs. one cluster.

In [19]:
df2.groupby('subject').agg(dict(cluster_id=['min', 'max'])).nunique(axis=1).value_counts()

1    20
2    13
dtype: int64